## Pressure sensor placement for leakage detection


### Initialize EPANET Python Toolkit (EPyT)

You should always begin with this command to import the toolkit.

[EPyT](https://github.com/OpenWaterAnalytics/EPyT) is available on [PyPI](https://pypi.org/project/epyt/) and can be installed via `pip install epyt`. To upgrade to the latest version if it's already installed, use `pip install --upgrade epyt`.

In [ ]:
%pip install epyt

In [ ]:
from epyt import epanet

### Water Network

In [ ]:
d=epanet('data/Hanoi.inp')

In [ ]:
nj = d.getNodeJunctionCount()
nn = d.getNodeCount()
reservInd = d.getNodeReservoirIndex()

### Calculate Sensitivity matrix
[1] Cugueró , M., Puig, V., Quevedo, J. Optimal pressure sensor placement and assessment for leak location using a relaxed isolation index: Application to the Barcelona water network, Control Eng. Pract., vol. 63, pp. 1-12, Jun. 2017.

##### Calculate healthy states in extended time simulation

In [ ]:
import numpy as np

d.setTimeSimulationDuration(24*60*60)  # greater weight to low demand hours
d.setTimePatternStart(0)  # in seconds
allParameters = d.getComputedTimeSeries()
P0 = allParameters.Pressure.transpose()     

reservoir_indices = d.getNodeReservoirIndex()
for index in reservoir_indices:
    P0 = np.delete(P0, index-1, 0)

Dem0 = allParameters.Demand.transpose()     
simSteps = P0.shape[1]

# Create Augmented-time Sensitivity Matrix
# Simulate all leakages and get all scenarios pressures
leak_mag_desir = np.mean(np.mean(Dem0[Dem0 > 0]))       
mean_pressure = np.mean(np.mean(P0[P0 > 0]))            

leak_emit = leak_mag_desir / np.sqrt(mean_pressure)     
emit0 = d.getNodeEmitterCoeff()                         

S = np.zeros((nj, nj))      

for leak in range(nj):                  
    print('Calculating Sensitivity Matrix...')
    print('Simulating leakage', leak + 1, 'out of', nj)
    emit = np.zeros_like(emit0)
    emit[leak] = leak_emit  # set emitter coefficient (leakage) value   
    
    d.setNodeEmitterCoeff(emit)
    allParameters = d.getComputedTimeSeries()
    P = allParameters.Pressure.transpose()          
    
    reservoir_indices = d.getNodeReservoirIndex()
    for index in reservoir_indices:
        P = np.delete(P, index-1, 0)
        
    Dem = allParameters.Demand.transpose()         
    leak_mag = Dem[leak-1, :] - Dem0[leak-1, :]       
    
    Stmp = (P - P0) / leak_mag                      
    
    rmax = np.max(np.abs(Stmp), axis=0)
    Stmp = np.abs(Stmp) / rmax
    
    S[:, leak] = np.max(Stmp, axis=1) 

print('Calculations done!')



## Place sensors using GA
Select number of sensors to be installed:

In [ ]:
sensors_num=4

Define existing sensors in the network

In [ ]:
exist_sens_ind=[]; # define existing sensors in the network
# exist_sens_ind=d.getNodeIndex({'14','63','114','399','572','302','662'});

Solve using GA: Maximize the minimum sensitivity of all sensors to any leakage

#### Plot sensors

#### Cost function

#### Enumeration loops for threshold and sensors

In [ ]:
%pip install pymoo

In [ ]:
import numpy as np
from pymoo.core.problem import Problem
from pymoo.algorithms.soo.nonconvex.ga import GA
from pymoo.optimize import minimize

# Placeholder for your sensitivity matrix S and other variables
S = np.random.rand(100, 100)  # Placeholder for sensitivity matrix
nj = S.shape[1]  # Number of junctions

# Define the cost function
def GACostFunction(u, S):
    Sm = S[u > 0, :]
    Smax = np.max(Sm, axis=0)
    cost = -np.min(Smax)  # Maximize the minimum sensitivity
    return cost

# Define the GA optimization problem
class SensorPlacementProblem(Problem):
    def __init__(self, S, nj, sensors_num, exist_sens_ind):
        self.S = S
        self.nj = nj
        self.sensors_num = sensors_num
        self.exist_sens_ind = exist_sens_ind
        super().__init__(n_var=nj, n_obj=1, n_constr=0, xl=0, xu=1, type_var=int)

    def _evaluate(self, x, out, *args, **kwargs):
        f = np.array([GACostFunction(xi, self.S) for xi in x])
        out["F"] = f

# Define the problem
problem = SensorPlacementProblem(S, nj, sensors_num, exist_sens_ind)

# Configure the Genetic Algorithm (GA)
algorithm = GA(
    pop_size=200,            # Population size
    sampling=None,           # Not specified, can be LHS or other sampling methods
    crossover="real_one_point",  # Crossover method
    mutation="real_pm",      # Mutation method
)

# Solve the optimization problem
res = minimize(problem,
               algorithm,
               ('n_gen', 100),  # Number of generations
               seed=1,
               verbose=True)

# Extract the result
u = res.X

# Get indices of top sensors
top_sensors_indices = np.argsort(u)[-sensors_num:][::-1]
print("Indices of top", sensors_num, "selected sensors:", top_sensors_indices)
